In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

import helper_functions as sc

In [131]:
results = pd.read_csv('data\\results_2017.csv', index_col=0)
teams = pd.read_csv('data\\teams_2017.csv', index_col=0, sep=';')
n_gameday = len(results['round'].unique())

In [132]:
results = (results.merge(teams.add_prefix('home_'), left_on='home', right_on='home_id', how='left')
                    .merge(teams.add_prefix('away_'), left_on='away', right_on='away_id', how='left')
                    .drop(['home_id', 'away_id'], axis=1))
results.head()

,round,home,away,goals_home,goals_away,match_id,points_home,points_away,home_team,home_team_web,away_team,away_team_web
0,Bundesliga - 1,157,168,3,1,11225,3,0,Bayern Munich,bayern,Bayer Leverkusen,leverkusen
1,Bundesliga - 1,167,162,1,0,11220,3,0,1899 Hoffenheim,hoffenheim,Werder Bremen,bremen
2,Bundesliga - 1,164,166,0,1,11221,0,3,FSV Mainz 05,mainz,Hannover 96,hannover
3,Bundesliga - 1,175,170,1,0,11222,3,0,Hamburger SV,hamburg,FC Augsburg,augsburg
4,Bundesliga - 1,159,172,2,0,11223,3,0,Hertha Berlin,herthabsc,VfB Stuttgart,stuttgart
5,Bundesliga - 1,161,165,0,3,11224,0,3,VfL Wolfsburg,wolfsburg,Borussia Dortmund,dortmund
6,Bundesliga - 1,174,173,2,0,11219,3,0,FC Schalke 04,schalke,RB Leipzig,rbleipzig
7,Bundesliga - 1,160,169,0,0,11218,1,1,SC Freiburg,freiburg,Eintracht Frankfurt,frankfurt
8,Bundesliga - 1,163,192,1,0,11217,3,0,Borussia Monchengladbach,mgladbach,FC Koln,koeln
9,Bundesliga - 2,192,175,1,3,11216,0,3,FC Koln,koeln,Hamburger SV,hamburg


In [201]:
stats = {}

for r in results['round'].unique():
    stats[r] = {}

for i in range(len(results)):
    gameday = results['round'][i][13:]
    # get url
    home = results['home_team_web'][i]
    away = results['away_team_web'][i]
    url = 'https://www.fussballdaten.de/bundesliga/2018/'+gameday+'/'+home+'-'+away+'/'
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find_all('div')
    status = divs[1].find_all('div', {'class': "statistik-reihe"})
    # get team id
    home = results['home'][i]
    away = results['away'][i]
    gameday = results['round'][i]
    # initialize dictionary
    stats[gameday][home] = {}
    stats[gameday][away] = {}    
    for row in status:
        column_name = row.find('div', {'class': "text-center"})
        column_name = (re.sub(clean, '', str(column_name)))
        values = row.find_all('span', {'class':'circle-statistik'})
        values = [(re.sub(clean, '', str(value))) for value in values]
        stats[gameday][home][column_name] = values[0]
        stats[gameday][away][column_name] = values[1]

{'Bundesliga - 1': {157: {'Tore': '3',
   'Ballbesitz (%)': '50',
   'Schüsse aufs Tor': '7',
   'Schüsse neben das Tor': '5',
   'Freistöße': '17',
   'Eckbälle': '4',
   'Abseits': '3',
   'Gehaltene Bälle': '4',
   'Fouls': '13',
   'ø-Note': '2.8',
   'Zweikämpfe (%)': '51,9',
   'Pässe (%)': '85,9',
   'Gelbe Karten': '1',
   'Platzverweise': '0'},
  168: {'Tore': '1',
   'Ballbesitz (%)': '50',
   'Schüsse aufs Tor': '5',
   'Schüsse neben das Tor': '10',
   'Freistöße': '16',
   'Eckbälle': '5',
   'Abseits': '1',
   'Gehaltene Bälle': '5',
   'Fouls': '16',
   'ø-Note': '3.6',
   'Zweikämpfe (%)': '48,1',
   'Pässe (%)': '86,4',
   'Gelbe Karten': '2',
   'Platzverweise': '0'},
  167: {'Tore': '1',
   'Ballbesitz (%)': '57',
   'Schüsse aufs Tor': '3',
   'Schüsse neben das Tor': '8',
   'Freistöße': '17',
   'Eckbälle': '6',
   'Abseits': '3',
   'Gehaltene Bälle': '2',
   'Fouls': '16',
   'ø-Note': '3.1',
   'Zweikämpfe (%)': '50,5',
   'Pässe (%)': '89,1',
   'Gelbe Karten'

In [ ]:
stats

In [167]:
rounds = results['round'].unique()
table = {r:{t:{'points':[], 'goals_shot':[], 'goals_received':[]} for t in teams['id']} for r in rounds}

In [183]:
# setup table with base results
last_round = ''
for r in rounds:
    for t in teams['id']:
        try:
            match_id = int(results[(results['round'] == r) & (results['home'] == t)]['match_id'])
            points = int(results[(results['round'] == r) & (results['home'] == t)]['points_home'])
            goals_shot = int(results[(results['round'] == r) & (results['home'] == t)]['goals_home'])
            goals_received = int(results[(results['round'] == r) & (results['home'] == t)]['goals_away'])
        except:
            match_id = int(results[(results['round'] == r) & (results['away'] == t)]['match_id'])
            points = int(results[(results['round'] == r) & (results['away'] == t)]['points_away'])
            goals_shot = int(results[(results['round'] == r) & (results['away'] == t)]['goals_away'])
            goals_received = int(results[(results['round'] == r) & (results['away'] == t)]['goals_home'])
        table[r][t]['points'] = points
        if last_round == '':
            table[r][t]['points_cum'] = points
        else:
            table[r][t]['points_cum'] = table[last_round][t]['points_cum'] + points
            goals_shot = table[last_round][t]['goals_shot'] + goals_shot
            goals_received = table[last_round][t]['goals_received'] + goals_received
        table[r][t]['goal_difference'] = table[r][t]['goals_shot']-table[r][t]['goals_received']
    last_round = r
#table

In [191]:
# get rank & rating
for r in rounds:
    gameday = pd.DataFrame()
    for t in teams['id']:
        gameday = gameday.append(pd.DataFrame({'round': [r],
                                               'team': [t],
                                               'points_cum': table[r][t]['points_cum'],
                                               'goal_difference': table[r][t]['goal_difference']}))
    gameday = gameday.sort_values(['points_cum', 'goal_difference'], ascending = False)
    gameday['rank'] = [i for i in range(1,len(gameday)+1)]
    for t in teams['id']:
        table[r][t]['rank'] = int(gameday[gameday['team'] == t]['rank'])
        table[r][t]['rating'] = sc.get_rating(int(table[r][t]['rank']))

In [193]:
# get form
for t in teams['id']:
    form = []
    form_rating = []
    for r in rounds:
        if len(form) > 4:
            form.pop(0)
            form_rating.pop(0)
        form.append(table[r][t]['points'])
        form_rating.append(table[r][t]['rating'])
        table[r][t]['form'] = form.copy()
        table[r][t]['form_rating'] = form_rating.copy()

In [227]:
stats

{'Bundesliga - 1': {157: {'Tore': '3',
   'Ballbesitz (%)': '50',
   'Schüsse aufs Tor': '7',
   'Schüsse neben das Tor': '5',
   'Freistöße': '17',
   'Eckbälle': '4',
   'Abseits': '3',
   'Gehaltene Bälle': '4',
   'Fouls': '13',
   'ø-Note': '2.8',
   'Zweikämpfe (%)': '51,9',
   'Pässe (%)': '85,9',
   'Gelbe Karten': '1',
   'Platzverweise': '0'},
  168: {'Tore': '1',
   'Ballbesitz (%)': '50',
   'Schüsse aufs Tor': '5',
   'Schüsse neben das Tor': '10',
   'Freistöße': '16',
   'Eckbälle': '5',
   'Abseits': '1',
   'Gehaltene Bälle': '5',
   'Fouls': '16',
   'ø-Note': '3.6',
   'Zweikämpfe (%)': '48,1',
   'Pässe (%)': '86,4',
   'Gelbe Karten': '2',
   'Platzverweise': '0'},
  167: {'Tore': '1',
   'Ballbesitz (%)': '57',
   'Schüsse aufs Tor': '3',
   'Schüsse neben das Tor': '8',
   'Freistöße': '17',
   'Eckbälle': '6',
   'Abseits': '3',
   'Gehaltene Bälle': '2',
   'Fouls': '16',
   'ø-Note': '3.1',
   'Zweikämpfe (%)': '50,5',
   'Pässe (%)': '89,1',
   'Gelbe Karten'

In [225]:
# get match statistics
for t in teams['id']:
    form_posession = []
    form_duels = []
    form_passes = []
    form_accuracy = []
    form_offense = []
    for r in rounds:
        if len(form_posession) > 4:
            form_posession.pop(0)
            form_duels.pop(0)
            form_passes.pop(0)
            form_accuracy.pop(0)
            form_offense.pop(0)
        form_posession.append(int(stats[r][t]['Ballbesitz (%)'][:2]))
        form_duels.append(int(stats[r][t]['Zweikämpfe (%)'][:2]))
        form_passes.append(int(stats[r][t]['Pässe (%)'][:2]))
        accuracy = round(int(stats[r][t]['Tore'])/(int(stats[r][t]['Schüsse aufs Tor'])+0.00001),2)
        form_accuracy.append(accuracy)
        form_offense.append(int(stats[r][t]['Tore']))
        table[r][t]['form_possession'] = form_posession.copy()
        table[r][t]['form_duels'] = form_duels.copy()
        table[r][t]['form_passes'] = form_passes.copy()
        table[r][t]['form_accuracy'] = form_accuracy.copy()
        table[r][t]['form_offense'] = form_offense.copy()

In [245]:
data = pd.DataFrame()
for i in range(len(results)):
    team_1 = results.loc[i]['home']
    team_2 = results.loc[i]['away']
    gameday = results.loc[i]['round']
    if gameday in ['Bundesliga - 1', 'Regular Season - 1']:
        last_gameday = gameday
        continue
    data = data.append(sc.get_sample_row(gameday, last_gameday, team_1, team_2, results, i, table, 1))
    data = data.append(sc.get_sample_row(gameday, last_gameday, team_2, team_1, results, i, table, 0))
    last_gameday = gameday
data = data.reset_index(drop=True)
data.head()

,round,team_1,goal_difference_1,rating_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,form_duels_1,...,form_2,form_weighted_2,form_possession_2,form_pass_acc_2,form_shot_acc_2,form_duels_2,form_offense_2,home_away,target,target_possession
0,Bundesliga - 2,192,-1,C,0.0,0.0,48.0,84.0,0.000,50.0,...,3.0,0.0,39.0,70.0,0.200,49.0,1.0,1,0,84
1,Bundesliga - 2,175,1,C,3.0,6.0,39.0,70.0,0.200,49.0,...,0.0,0.0,48.0,84.0,0.000,50.0,0.0,0,0,70
2,Bundesliga - 2,168,-2,C,0.5,1.0,54.5,86.5,0.435,54.0,...,2.0,0.5,49.0,83.0,0.415,44.5,1.5,1,1,87
3,Bundesliga - 2,167,1,B,2.0,6.0,49.0,83.0,0.415,44.5,...,0.5,3.0,54.5,86.5,0.435,54.0,1.5,0,1,77
4,Bundesliga - 2,169,-1,C,0.5,1.0,45.0,76.0,0.000,52.0,...,1.5,0.5,44.5,68.5,0.165,49.5,0.5,1,0,76


In [250]:
data.to_csv('data\\bundesliga_2017_full.csv')

In [251]:
team_1 = pd.get_dummies(data['team_1'], prefix = 'team_1_', drop_first = True)
team_2 = pd.get_dummies(data['team_2'], prefix = 'team_2_', drop_first = True)
rating_1 = pd.get_dummies(data['rating_1'], prefix = 'team_1', drop_first = True)
rating_2 = pd.get_dummies(data['rating_2'], prefix = 'team_2', drop_first = True)

In [254]:
data_ready = (data.join(rating_1)
                  .join(rating_2)
                  .drop(['rating_1', 'rating_2'], axis=1))
data_ready.head()

,round,team_1,goal_difference_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,form_duels_1,form_offense_1,...,form_offense_2,home_away,target,target_possession,team_1_B,team_1_C,team_1_D,team_2_B,team_2_C,team_2_D
0,Bundesliga - 2,192,-1,0.0,0.0,48.0,84.0,0.000,50.0,0.0,...,1.0,1,0,84,0,1,0,0,1,0
1,Bundesliga - 2,175,1,3.0,6.0,39.0,70.0,0.200,49.0,1.0,...,0.0,0,0,70,0,1,0,0,1,0
2,Bundesliga - 2,168,-2,0.5,1.0,54.5,86.5,0.435,54.0,1.5,...,1.5,1,1,87,0,1,0,1,0,0
3,Bundesliga - 2,167,1,2.0,6.0,49.0,83.0,0.415,44.5,1.5,...,1.5,0,1,77,1,0,0,0,1,0
4,Bundesliga - 2,169,-1,0.5,1.0,45.0,76.0,0.000,52.0,0.0,...,0.5,1,0,76,0,1,0,0,1,0


In [255]:
data_ready.to_csv('data\\bundesliga_2017_ready.csv')